In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [2]:
import os
import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Uncomment the following line if you need to initialize FAISS with no AVX2 optimization
# os.environ['FAISS_NO_AVX2'] = '1'

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [4]:
from langchain.docstore.document import Document

def read_tags_file(file_path):
    with open(file_path, 'r', errors='ignore') as file:
        lines = file.readlines()

    tags = []
    for line in lines:
        if line.startswith('!'):  # Skip metadata lines
            continue
        parts = line.split('\t')
        if len(parts) >= 4:
            tag_name = parts[0]
            file_name = parts[1]
            pattern = parts[2]
            tags.append(dict(tag_name=tag_name, file_name=file_name, pattern=pattern))

    return tags

# Use the function
tags = read_tags_file('./repo/langchain/libs/langchain/tags')

documents = []

for tag in tags:
    documents.append(Document(page_content=f"{tag['file_name']} | {tag['tag_name']} ", metadata=tag))


In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
db = FAISS.from_documents(documents, embeddings)

In [7]:
query = "How can i add a Custom Prompt Template in this repository?"
docs = db.similarity_search(query, k=5)

In [8]:
docs

[Document(page_content='langchain/prompts/prompt.py | from_template ', metadata={'tag_name': 'from_template', 'file_name': 'langchain/prompts/prompt.py', 'pattern': '/^    def from_template(cls, template: str, **kwargs: Any) -> PromptTemplate:$/;"'}),
 Document(page_content='langchain/prompts/prompt.py | PromptTemplate ', metadata={'tag_name': 'PromptTemplate', 'file_name': 'langchain/prompts/prompt.py', 'pattern': '/^class PromptTemplate(StringPromptTemplate):$/;"'}),
 Document(page_content='langchain/schema/prompt_template.py | _prompt_type ', metadata={'tag_name': '_prompt_type', 'file_name': 'langchain/schema/prompt_template.py', 'pattern': '/^    def _prompt_type(self) -> str:$/;"'}),
 Document(page_content='langchain/schema/prompt_template.py | BasePromptTemplate ', metadata={'tag_name': 'BasePromptTemplate', 'file_name': 'langchain/schema/prompt_template.py', 'pattern': '/^class BasePromptTemplate(Serializable, ABC):$/;"'}),
 Document(page_content='langchain/schema/prompt_templa

In [9]:

import inspect
import importlib.util
import os

root = "repo/langchain/libs/langchain"
 

def get_source_code(function_name, function_path):
    spec=importlib.util.spec_from_file_location(function_name, os.path.join(root, function_path))
    foo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(foo)
    return inspect.getsource(foo)



In [10]:
template = """You are a hupful bot that fuilfill the human' program task:

The following is releative code:
{code_file_text}

User: {user_prompt}
Ai:"""

In [11]:
from typing import List
def create_code_file_text(docs : List[Document]):
    code_file_text = ""
    for i, doc in enumerate(docs):
        code_file_text += f'==== File {i+1}/{len(docs)} ====\n'
        code_file_text += f'File path: {doc.metadata["file_name"]}\n'
        code_file_text += f'Tag name: {doc.metadata["tag_name"]}\n'
        code_file_text += f'Code: {get_source_code(doc.metadata["tag_name"], doc.metadata["file_name"])}\n'
        code_file_text += "\n"
    return code_file_text
    

In [12]:
code_text = create_code_file_text(docs)

In [13]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain.agents import AgentType


# llm = AzureChatOpenAI(
#     openai_api_key=getpass.getpass("OpenAI API Key:"),
#     deployment_name="gpt35-chat",
#     openai_api_type="azure",
#     openai_api_base=getpass.getpass("OpenAI API Base:"),
#     openai_api_version="2023-03-15-preview",
# )

In [16]:
def ask(user_prompt):
    # find docs similar to user_prompt
    docs = db.similarity_search(query, k=5)
    result = template.format(code_file_text=code_text, user_prompt=user_prompt)
    # call openai api here
    return result

In [17]:
ask("How can i add a Custom Prompt Template in this repository?")

'You are a hupful bot that fuilfill the human\' program task:\n\nThe following is releative code:\n==== File 1/5 ====\nFile path: langchain/prompts/prompt.py\nTag name: from_template\nCode: """Prompt schema definition."""\nfrom __future__ import annotations\n\nfrom pathlib import Path\nfrom string import Formatter\nfrom typing import Any, Dict, List, Union\n\nfrom pydantic import root_validator\n\nfrom langchain.prompts.base import (\n    DEFAULT_FORMATTER_MAPPING,\n    StringPromptTemplate,\n    _get_jinja2_variables_from_template,\n    check_valid_template,\n)\n\n\nclass PromptTemplate(StringPromptTemplate):\n    """Schema to represent a prompt for an LLM.\n\n    Example:\n        .. code-block:: python\n\n            from langchain import PromptTemplate\n            prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}")\n    """\n\n    @property\n    def lc_attributes(self) -> Dict[str, Any]:\n        return {\n            "template_format": self.template_format,\n  